# Энкодер-декодер архитектура
Сегодня мы рассмотрим такую важную архитектуру как энкодер-декодер.
В контексте обработки естественного языка эта архитекутра используется чаще всего в задачах преобразования последовательности в последовательность (seq2seq). Такие задачи включают, например, машинный перевод. 

Вопрос: Что еще бывает?

Рассмотрим общий вид таких моделей (здесь и далее иллюстрации из курса Лены Войты): 

![alt_text](../../additional_materials/images/enc_dec-min.png)

Мы с вами до сих пор занимались рекуррентными сетями. Мы уже знаем, как модель работает в случае обычного моделирования языка. В случае энкодер-декодера в ней принципиально ничего не меняется - мы все еще будем генерировать предложения токен за токеном. Однако для формирования результата нам нужно уже что-то, описывающее вход. Энкодер-декодер в случае перевода имеет вид что=то вроде следующего:

![alt_text](../../additional_materials/images/enc_dec_simple_rnn-min.png)

Идея довольно проста - давайте использовать последнее скрытое состояние закодированного входа как начальное для выхода. 
Вопрос: в чем минус такого решения?
Вопрос: а если у нас на входе не текст?


Мы опробуем наши модели кодировщика-декодера на проблеме понимания изображений. Эта задача, с одной стороны, заставляет нас использовать соответствующую архитектуру, с другой - позволяет опустить обучение энкодера, сосредоточившись только на декодере. 

Прежде чем мы перейдем к архитектуре, необходимо выполнить предварительную обработку. У нас есть две части нашей задачи - изображения и их описания. Для текстов нам необходимо применить токенизацию.

Однако, в отличие от прошлого раза, мы не будем применять простейшее представление текста. Наши строки данных содержат уникальные редкие слова. Если мы будем действовать на уровне слова, нам придется иметь дело с большим словарным запасом, мучаться с нормализацией и т.д. Если вместо этого использовать модели на уровне символов, то для обработки последовательности потребуется много итераций. На этот раз мы выберем что-нибудь среднее.

Один из популярных подходов называется кодированием Byte Pair (BPE). Алгоритм начинается с токенизации на уровне символов, а затем итеративно объединяет наиболее часто встречающиеся пары в течении N итераций. Это приводит к тому, что часто встречающиеся слова объединяются в один символ, а редкие слова разбиваются на слоги или даже символы. С одной стороны, мы отнсительно эффективно составляем словарб, с другой стороны, если мы даже не будем знать новое слово, мы сможем побить его на символы и все равно закодировать.

Установим необходиме библиотеки:

In [ ]:
!pip install nltk

In [ ]:
!pip install subword_nmt

In [ ]:
from os.path import join as pjoin
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
import os
import pandas as pd
from PIL import Image
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from collections import Counter

from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.distributions.categorical import Categorical
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights


In [ ]:
data_path = "../../datasets/flikr"

Протокенизируем наш датасет. Здесь мы возьмем все текстовые строки из корпуса и составим токены для них.
Материал по токенизации: [Byte-Pair Encoding (BPE)](https://huggingface.co/learn/nlp-course/ru/chapter6/5)

Вопрос: а как нам пришлось бы токенизировать тексты в случае машинного перевода?

In [ ]:
tokenizer = WordPunctTokenizer()

def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

# разделение описаний картинок на отдельные токены
with open('train', 'w') as f_src:
    for line in open(pjoin(data_path, 'captions.txt')):
        words = line.strip().split(',')
        image, src_line = words[0], ",".join(words[1:])
        f_src.write(tokenize(src_line) + '\n')

# build and apply bpe voc
learn_bpe(open('./train'), open('bpe_rules', 'w'), num_symbols=8000)
bpe = BPE(open('./bpe_rules'))

with open('train.bpe', 'w') as f_out:
    for line in open('train'):
        f_out.write(bpe.process_line(line.strip()) + '\n')

Теперь нам нужно создать словарь, который сопоставляет строки с токенами и наоборот.Это нам понадобится, когда мы захотим тренировать модель.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_inp = np.array(open('./train.bpe').read().split('\n'))
from vocab import Vocab
voc = Vocab.from_lines(data_inp)

Наш токенизатор и словарь умеют переводить сразу несколько строк в матрицу токенов, давая результирующий тензор максимальной или заданной длины. Проверим, что все нормально.

In [ ]:
# Here's how you cast lines into ids and backwards.
batch_lines = sorted(data_inp, key=len)[5:10]
batch_ids = voc.to_matrix(batch_lines)
batch_lines_restored = voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

Также мы можем узнать, какое распределение числа токенов на предложение

In [ ]:
plt.figure(figsize=[8, 4])
plt.title("caption length")
plt.hist(list(map(len, map(str.split, data_inp))), bins=20);

Как выглядит наш датасет? 

In [ ]:
captions = pd.read_csv(pjoin(data_path, 'captions.txt')).dropna()
captions

In [ ]:
#captions["caption"][0]

# Here's how you cast lines into ids and backwards.
batch_lines = sorted(data_inp, key=len)[5:10] # captions["caption"][0]
batch_ids = voc.to_matrix(batch_lines)
batch_lines_restored = voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

In [ ]:
image_file = captions["image"].sample(1).iloc[0]
# выведите все описания для этой картинки
Image.open(pjoin(data_path, "Images", image_file))

In [ ]:
image_ids = {k: i for i, k in enumerate(captions["image"].unique())}
image_list = list(map(lambda x: x[0], sorted(image_ids.items(), key=lambda x: x[1])))
captions['image_id'] = captions["image"].map(image_ids)

In [ ]:
from PIL import Image
#from torchvision.io import read_image

class ImagesDataset(Dataset):
    def __init__(self, root, image_list, transform=None):
        super().__init__()
        self.root = root
        self.image_list = image_list
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, item):
        # просто загрузите и трансформируйте картинку
        image_path = self.image_list[item]
        #image = read_image(self.root + "/" + image_path)
        image = Image.open(self.root + "/" + image_path)
        
        if self.transform is not None:
            image = self.transform(image)

        return image

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from torch import nn
cnn_model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1).eval().to(device)
cnn_model.classifier = nn.Identity()

In [ ]:
import torchvision.transforms as T

transform = T.Compose([
    T.ToTensor(),
    T.Resize(256),
    T.CenterCrop(224),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImagesDataset(root=pjoin(data_path, "Images"), image_list=image_list, transform=transform)
dataloader = DataLoader(dataset, shuffle=False, batch_size=4)

In [ ]:
plt.imshow(dataset[0].permute(1, 2, 0))

In [ ]:
#!pip install ipywidgets

In [ ]:
image_embeds = []

with torch.no_grad():
    for images in tqdm(dataloader):
        embeds = cnn_model(images.to(device))
        image_embeds += [embeds.cpu()]

image_embeds = torch.cat(image_embeds, dim=0)

Протестируем результат

In [ ]:
i = 1001
image = Image.open(pjoin(data_path, "Images", image_list[i])).convert('RGB')
image

In [ ]:
cnn_model(transform(image).to(device).unsqueeze(0))

In [ ]:
image_embeds[i]

In [ ]:
captions[captions.image_id == 1001]

Итак, у нас все готово для создания тренировочного датасета: вхожные изображения, эмбеддинги и описания. При этом несколько описаний могут использоваться для одного изображения, поэтому будем делить датасет именно по описаниям.

In [ ]:
from sklearn.model_selection import train_test_split

#train_images, test_images, train_embeds, test_embeds =  # разделите датасет


train_embeds, test_embeds = train_test_split(image_embeds, shuffle=False)
train_images = test_images = image_list

train_embeds.shape, test_embeds.shape

In [ ]:
train_captions = captions[captions["image"].isin(train_images)]
test_captions = captions[captions["image"].isin(test_images)]

train_captions.shape, test_captions.shape

In [ ]:
class CaptionsDataset(Dataset):
    def __init__(self, captions, embeds, images, max_len=64):
        super().__init__()
        self.captions = captions.reset_index()
        self.images = images
        self.embeds = embeds

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, item):
        image_id = self.captions.loc[item, "image_id"]
        return { "x": self.embeds[image_id], "y": self.captions.loc[item, "caption"]}

В этот раз мы попробуем написать свою кастомную функцию склейки в батч с использованием словаря. Она берет элементы батча (список словарей)) и должна объединить x и y в тензоры

In [ ]:
def rnn_collate_fn(batch):
    # your code 
    return {
        "x": torch.stack([batch_i['x'] for batch_i in batch], dim=0), #
        "y": [batch_i['y'] for batch_i in batch]  #
    }

Теперь ее достаточно передать в даталоадер при инициализации

In [ ]:
train_set = CaptionsDataset(train_captions, image_embeds, image_list)
test_set = CaptionsDataset(test_captions, image_embeds, image_list)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, collate_fn=rnn_collate_fn)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, collate_fn=rnn_collate_fn)
embeds_loader = DataLoader(test_embeds, batch_size=128, shuffle=False, collate_fn=rnn_collate_fn)

In [ ]:
train_set[0]

In [ ]:
for batch in train_loader:
    break

In [ ]:
batch

Теперь займемся моделью. Она должна содержать как энкодер, так и декодер. Энкодер должен преобразовывать входной эмбеддинг в понятное модели начальное состояние (скрытое представление и вход), а декодер должен, начиная с этого состояния, генерировать последовательность. Обратите внимание, что если слоев у LSTM больше одного, то инициализировать мы должны все слои сразу. 

Здесь мы также использоуем слой эмбеддинга, который позволит нам перевести токены в векторы.


In [ ]:
class CaptionRNN(nn.Module):
    def __init__(self, image_embed_dim, vocab_size, pad_index=1, eos_index=-1, embed_dim=256, hidden_dim=256, lstm_layers=1, dropout=0.1):
        super().__init__()
        self.lstm_layers = lstm_layers
        self.hidden_dim = hidden_dim

        self.image_embed_to_h0 = # your code
        self.image_embed_to_c0 = # your code
        self.embedding = # your code
        self.lstm = # your code
        self.linear = # your code
        self.eos_index = eos_index
        self.pad_index = pad_index 

    def forward(self, tokens, image_embeds):
        '''
        B - batch size
        M - lstm layers
        L - sequence length
        I - image embedding dim
        E - embedding dim
        H - hidden dim
        V - vocab size
        '''
        # image_embeds: (B, I)
        B = # your code
        h0 = # your code
        c0 = # your code
        # h0, co: (M, B, H)

        # tokens: (B, L)
        # embeds: (B, L, E)
        # output: (B, L, H), (h, c): (M, B, H)
        # logits: (B, L, V)
        return logits

    @torch.no_grad()
    def inference(self, image_embeds):
        self.eval()
        # generate lstm input
        B = image_embeds.shape[0]
        h = # your code
        c = # your code
        h, c = h.contiguous(), c.contiguous()

        # init tokens with <bos>
           # your code
        # 2 stopping conditions: reaching max len or getting <eos> token
        while tokens.shape[1] < 64:
            if ((tokens == self.eos_index).sum(1) > 0).all():
                break

            # process newly obtained token
            # your code
            logits = # your code

            # get new tokens from logits
            new_tokens = logits.argmax(dim=-1)
            tokens = torch.cat([tokens, new_tokens], dim=1)

        return tokens

In [ ]:
model = CaptionRNN(image_embeds.shape[1], vocab_size=len(voc), eos_index=voc.eos_ix, pad_index=voc.unk_ix)

for batch in train_loader:
    break  

In [ ]:
batch

In [ ]:
# testing

logits = model(batch["y"], batch["x"])

tokens = model.inference(embeds)
voc.to_lines(tokens)

In [ ]:
logits.shape

In [ ]:
from IPython.display import clear_output
from tqdm.notebook import tqdm
import numpy as np


BLEU_FREQ = 5


def plot_losses(train_losses, test_losses, test_blues):
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(13, 4))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train', color='deepskyblue', linewidth=2)
    axs[0].plot(range(1, len(test_losses) + 1), test_losses, label='test', color='springgreen', linewidth=2)
    axs[0].set_ylabel('loss')

    axs[1].plot(BLEU_FREQ * np.arange(1, len(test_blues) + 1), test_blues, label='test',
                color='springgreen', linewidth=2)
    axs[1].set_ylabel('BLEU')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    plt.show()

Напишем трейн луп. Он похож на то, что мы делали раньше, однако мы не используем пакинг. Допольнительное задание: напишите лосс функцию для нашей задачи вручную.

Для оценки модели мы попробуем использовать BLEU.Эта метрика была создана для машинного перевода, но может использоваться и длядругих приложений. Эта метрика просто вычисляет, какая часть предсказанных n-грамм действительно присутствует в эталонном переводе. Он делает это для n=1,2,3 и 4 и вычисляет среднее геометрическое со штрафом, если перевод короче эталонного.

Хотя BLEU имеет множество недостатков, он по-прежнему остается наиболее часто используемой метрикой и одной из самых простых для расчета.

In [ ]:
from torchmetrics import BLEUScore


def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc):
    train_loss = 0.0
    model.train()
    for batch in tqdm(train_loader, desc=tqdm_desc):
        # your code
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * embeds.shape[0]

    train_loss /= len(train_loader.dataset)
    return train_loss


@torch.no_grad()
def validation_epoch(model, criterion, valid_loader, tqdm_desc):
    valid_loss = 0.0
    model.eval()
    for batch in tqdm(valid_loader, desc=tqdm_desc):
        # your code
        valid_loss += loss.item() * embeds.shape[0]

    valid_loss /= len(valid_loader.dataset)
    return valid_loss


def evaluate_bleu(model, embeds_loader):
    bleu = BLEUScore()
    # your code
    return bleu(# your code).item()


def train(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs, log_frequency=1):
    train_losses, valid_losses, valid_blues = [], [], []

    for epoch in range(1, num_epochs + 1):
        train_loss = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}'
        )
        valid_loss = validation_epoch(
            model, criterion, valid_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}'
        )

        if epoch % log_frequency == 0:
            valid_bleu = evaluate_bleu(model, valid_loader)
            valid_blues += [valid_bleu]

        if scheduler is not None:
            scheduler.step()

        train_losses += [train_loss]
        valid_losses += [valid_loss]
        plot_losses(train_losses, valid_losses, valid_blues)

In [ ]:
NUM_EPOCHS = 200

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = None
criterion = nn.CrossEntropyLoss(ignore_index=voc.unk_ix)

sum(param.numel() for param in model.parameters())

In [ ]:
train(model, optimizer, scheduler, criterion, train_loader, test_loader,5)

Также посмотрим на случайную картинку из нашего набора.

In [ ]:
def caption_random_test_image():
    index = np.random.randint(len(test_images))
    image_file = test_images[index]
    tokens = model.inference(image_embeds[index].unsqueeze(0).to(device)).cpu()
    prediction = voc.to_lines(tokens)
    print('Prediction:', prediction)

    for i, caption in enumerate(captions[captions["image"] == image_file].caption):
        print(f'GT caption #{i + 1}:', caption)

    return Image.open(pjoin(data_path, 'Images', image_file)).convert('RGB')

In [ ]:
caption_random_test_image()

Что дальше? 

- Вы можете использовать механизм внимания, чтобы модель была лучше интерпреируема и была качественней.
Как это работает: https://distill.pub/2016/augmented-rnns/
Один из способов сделать это: https://arxiv.org/abs/1502.03044.
- Можно перейти на трансформеры